# <center> Введение

Внутренние (параметры модели): Подбираются во время обучения и определяют, как использовать входные данные для получения необходимого результата. Например, это веса (коэффициенты уравнения) в линейной/логистической регрессии.
Внешние (параметры алгоритма): Их принято называть гиперпараметрами. Внешние параметры могут быть произвольно установлены перед началом обучения и контролируют внутреннюю работу обучающего алгоритма. Например, это параметр регуляризации в линейной/логистической регрессии.

Тщательный подбор гиперпараметров гарантирует, что модель покажет максимально возможную точность на обучающих данных, но это совершенно не означает хороший результат на тестовых или новых данных.

# <center> 2. Базовая оптимизация

В базовой оптимизации, предоставляемой библиотекой sklearn, есть два основных метода — grid search и random search.

поиск по сетке (grid search) это попытка перебрать все возможные комбинации заданных гиперпараметров. Мы указываем список значений для различных гиперпараметров, и, ориентируясь на нашу метрику, оцениваем эффективность модели для каждого их сочетания, чтобы получить оптимальную комбинацию значений.

### ОПАСНОСТЬ ПЕРЕОБУЧЕНИЯ И УТЕЧКИ ДАННЫХ

разбить данные на три части: обучающую для построения модели, проверочную (валидационную) для выбора гиперпараметров модели, а также тестовую для оценки качества модели и выбранных гиперпараметров. 
Для лучшей оценки обобщающей способности вместо одного разбиения данных на обучающий и проверочный наборы мы можем воспользоваться перекрёстной проверкой, то есть кросс-валидацией (cross validation). 

### GRIDSEARCHCV

Основные параметры GridSearchCV:
- estimator — алгоритм, который будем оптимизировать;
- param_grid — словарь или список словарей. Словарь с именами гиперпараметров (в формате строки (str), например, 'max_depth') в качестве ключей и списками параметров (например, [5, 8, 10]) в качестве значений. Итого: {'max_depth': [5, 8, 10] }.
- scoring — по умолчанию используется score-функция заданного алгоритма: для классификации — sklearn.metrics.accuracy_score; для регрессии — sklearn.metrics.r2_score;
- cv — количество фолдов в кросс-валидации, по умолчанию используется 5.
- n_jobs — количество ядер для распараллеливания расчёта. -1 использует все существующие ядра.

*The choice of algorithm depends on the penalty chosen: Supported penalties by solver:*
'newton-cg' - ['l2', 'none']
'lbfgs' - ['l2', 'none']
'liblinear' - ['l1', 'l2']
'sag' - ['l2', 'none']
'saga' - ['elasticnet', 'l1', 'l2', 'none']

GridSearchCV включает в себя не только поиск лучших параметров, но и автоматическое построение новой модели на всём обучающем наборе данных с использованием параметров, которые дают наилучшее значение метрики при кросс-валидации.
Наилучшая найденная комбинация гиперпараметров сохраняется в атрибуте best_params_:
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))

Итоговая модель хранится в параметре best_estimator_, ей можно воспользоваться для получения прогнозов на новых данных:
print("Наилучшая модель:\n{}".format(grid_search.best_estimator_))

А наилучшее значение метрики на кросс-валидации (значение метрики, усреднённое по всем разбиениям для данной комбинации гиперпараметров) хранится в атрибуте best_score_. 
Примечание. Не путайте best_score_ со значением метрики модели, которое вычисляется на тестовом наборе с помощью метода score. Метод score (оценивающий качество результатов, полученных с помощью метода predict()) использует модель, построенную на всём обучающем наборе данных. В атрибуте best_score_ записывается средняя метрика на кросс-валидации.

Минус GRIDSEARCHCV: оценка всех возможных комбинаций требует множество времени и вычислительных ресурсов, а также кросс-валидации для обучения моделей.

### RANDOMIZEDSEARCHCV 

Рандомизированный поиск работает почти так же, как решётчатый поиск, мы можем задать количество подбираемых комбинаций, а не брать все возможные. RandomizedSearchCV выбирает n (количество задаем сами) случайных точек/комбинаций из заданных нами последовательностей. Как следствие, скорость сходимости к минимуму функции становится выше.

Основные параметры RandomizedSearchCV аналогичны GridSearchCV, за исключением n_iter:
- estimator — алгоритм, который будем оптимизировать;
- param_grid — словарь или список словарей. Словарь с именами гиперпараметров (в формате строки (str), например, 'max_depth') в качестве ключей и списками параметров (например, [5, 8, 10]) в качестве значений. Итого: {'max_depth': [5, 8, 10] }.
- scoring — по умолчанию используется score-функция заданного алгоритма: для классификации — sklearn.metrics.accuracy_score; для регрессии — sklearn.metrics.r2_score;
- cv — количество фолдов в кросс-валидации, по умолчанию используется 5.
- n_jobs — количество ядер для распараллеливания расчёта. -1 использует все существующие ядра.
- n_iter — количество комбинаций на расчёт. От этого параметра напрямую зависит время оптимизации и качество модели.

+ RandomizedSearchCV потребовалось меньше времени.

### РЕКОМЕНДАЦИИ ПО НАСТРОЙКЕ ГИПЕРПАРАМЕТРОВ АНСАМБЛЕЙ НАД РЕШАЮЩИМИ ДЕРЕВЬЯМИ

**АЛГОРИТМ СЛУЧАЙНОГО ЛЕСА (RANDOMFOREST)**
- n_estimators — число итераций (количество деревьев). Частично работает правило «чем больше, тем лучше», но иногда это не имеет особого смысла и сильно увеличивает затраты, поэтому стоит пробовать обучать сотни деревьев [100,200, 300, 400]. Если нет изменений, то оставить минимальное — 100.
- max_depth — максимальная глубина дерева. В случайном лесе строятся «сильные» деревья, каждое из которых даёт полноценный прогноз, поэтому глубина деревьем может быть достаточно большой. Стоит следить за переобучением.
- max_features — максимальное количество признаков, учитываемых алгоритмом при поиске лучшего разделения;
- subsample — доля выборки, которая будет использоваться для обучения каждого алгоритма — дерева.

**АЛГОРИТМ ГРАДИЕНТНОГО БУСТИНГА (GRADIENTBOOSTING)**
- n_estimators — число итераций (количество деревьев) : хотя ошибка на обучении монотонно стремится к нулю, ошибка на контроле, как правило, начинает увеличиваться после определенной итерации. Оптимальное число итераций можно выбирать, например, по отложенной выборке или с помощью кросс-валидации
- learning_rate — темп обучения (0;1]:
На практике оказывается, что градиентный бустинг очень быстро строит композицию, ошибка которой на обучении выходит на асимптоту (достигает предела), после чего начинает настраиваться на шум и переобучаться. Параметр learning_rate контролирует, насколько сильно каждое дерево будет пытаться исправить ошибки предыдущих деревьев. Более высокая скорость обучения означает, что каждое дерево может внести более сильные корректировки. Как правило, чем меньше темп обучения, тем лучше качество итоговой композиции.
- max_depth — максимальная глубина дерева. Используется для борьбы с переобучением. Рекомендуется устанавливать не более 5.
- max_features — максимальное количество признаков, учитываемых алгоритмом при поиске лучшего разделения.
- subsample — доля выборки, которая будет использоваться для обучения каждого алгоритма. Это ещё один способ улучшения качества градиентного бустинга. Таким образом вносится рандомизация в процесс обучения базовых алгоритмов, что снижает уровень шума в обучении, а также повышает эффективность вычислений. 
Рекомендация. Берите подвыборки, размер которых вдвое меньше исходной выборки.

Основные параметры градиентного бустинга деревьев — это количество деревьев (n_estimators) и скорость обучения (learning_rate), контролирующие степень вклада каждого дерева в устранение ошибок предыдущих деревьев. Эти два параметра тесно взаимосвязаны, поскольку более низкое значение learning_rate означает, что для построения модели аналогичной сложности необходимо большее количество деревьев.

В отличие от случайного леса, в котором более высокое значение n_estimators всегда дает лучшее качество, увеличение значения n_estimators в градиентном бустинге даёт более сложную модель, что может привести к переобучению. При всём этом случайный лес, в отличие от градиентного бустинга, использует глубокие деревья, способные сформировать полноценный прогноз. 

# <center> 3. Продвинутая оптимизация

Один из способов  — это байесовская оптимизация. Она отличается от случайного поиска или поиска по сетке тем, что учитывает предыдущие результаты, а не выбирает комбинации из вариантов, не имеющих информации о прошлых оценках. 

### TREE-STRUCTURED PARZEN ESTIMATORS (TPE)

1. На каждой итерации алгоритм TPE учитывает информацию о прошлых опробованных комбинациях гиперпараметров и только потом принимает решение, какой набор следует попробовать дальше. 
2. На следующем шаге происходит разделение собранных наборов на две группы: в первую группу входят наборы, дающие наилучшие результаты после оценки;
во вторую — все остальные.
Основная цель алгоритма — найти набор гиперпараметров, который с большей вероятностью будет в первой группе и с меньшей вероятностью во второй группе. Таким образом, для принятия следующего решения используется целое распределение наилучших комбинаций.
3. Далее TPE моделирует вероятности правдоподобия для каждой из групп, используя формулу Байеса:

p(x|y) = (p(x|y)*p(y))/p(x) где x — гиперпараметры, y — соответствующая оценка качества модели.

4. Затем, используя вероятность правдоподобия из первой группы, отбирается набор комбинаций, которые с большей вероятностью попадут в первую группу и с меньшей вероятностью — во вторую. 

5. Шаги 2-4  будет выполняться до тех пор, пока не будет достигнуто максимальное количество итераций. В итоге мы найдём наилучшую комбинацию гиперпараметров.

### HYPEROPT

это библиотека Python с открытым исходным кодом на основе байесовской оптимизации, в которой реализован алгоритм Tree-Structured Parzen Estimators (TPE).

Три шага для использования Hyperopt:
1. Задание пространства поиска гиперпараметров. Объявляем список гиперпараметров, тип распределения и его границы. Основные типы:
hp.choice(label, options) #равновероятный выбор из множестваю. использовать для категориальных
hp.randint(label, upper) #случайное целое число; random seed, например. использовать для целочисленных
hp.uniform(label, low, high) #равномерное непрерывное распределение. использовать для целочисленных и непрерывных
hp.normal(label, mu, sigma) #нормальное непрерывное распределение. использовать для непрерывных — аналогично целочисленным 
hp.lognormal(mu, sigma) #логнормальное непрерывное распределение. использовать для непрерывных — аналогично целочисленным 
2. Задание целевой функции. Создаём модель МО, передаём ей данные и оцениваем её на основе выбранной метрики. Можем минимизировать/максимизировать значение метрики.
3. Задание алгоритма поиска: Random Search или Tree of Parzen Estimators (TPE).


### OPTUNA

Три шага для использования Optuna: 
1. Задание пространства поиска гиперпараметров.
Основные функции:
suggest_categorical(name, choices) — для категориальных гиперпараметров;
suggest_int(name,low,high,step=1,log=False) — для целочисленных гиперпараметров;
suggest_float(name,low,high,step=None,log=False) — для непрерывных гиперпараметров;
suggest_uniform(name,low,high) — для целочисленных и непрерывных гиперпараметров.
2. Задание целевой функции. Создаём модель МО, передаём ей данные и оцениваем её на основе выбранной метрики, можем минимизировать/максимизировать значение метрики. 
3. Создание объекта исследования create study. 

### ПЛЮСЫ И МИНУСЫ РАССМОТРЕННЫХ ФРЕЙМВОРКОВ И БИБЛИОТЕК

GridSearchCV
+ Простой в использовании, идеален для небольшой сетки гиперпараметров
+ Встроенная кросс-валидация
+ Включает построение итоговой модели на всей обучающей выборке 
- Перебирает просто все комбинации заданной сетки гиперпараметров 
- Требует много времени и вычислительных ресурсов

RandomizedSearchCV
✔️ Эффективнее и экономичнее gridsearchcv
✔️ Можно задать количество рассматриваемых комбинаций
⛔️ Просто выбирает рандомные комбинации гиперпараметров без учёта результатов прошлых итераций

Hyperopt
✔️ Быстрее и эффективнее
✔️ Учитывает результаты прошлых итераций — байесовский оптимизатор
✔️ Можно отслеживать дополнительную информацию на каждом шаге с помощью класса Trials
✔️ Возможность построения условного пространства поиска гиперпараметров и даже моделей
⛔️ Старая документация, плохо с поддержкой и обновлением 
⛔️ Способен только минимизировать
⛔️ Непростой синтаксис описания пространства гиперпараметров (особенно с использованием условных реализаций)

Optuna
✔️ Один из самых быстрых и эффективных
✔️ Специально разработан для оптимизации гиперпараметров 
✔️ Простой в использовании
✔️ Относительно новый фреймворк с хорошей документацией 
✔️ Учитывает результаты прошлых итераций — байесовский оптимизатор 
✔️ Встроенная визуализация результатов
✔️ Возможность явно задавать максимизацию или минимизацию функции качества
⛔️ Удаление «плохих» точек пространства из рассмотрения
⛔️ Не стоит полностью полагаться: важно с умом определять пространство поиска, что может значительно сократить время расчётов.

# Выводы

Важно также помнить, что лучшие гиперпараметры модели на обучающей выборке не означают, что они будут лучшими и на тестовой выборке! Есть шанс переобучения, поэтому лучше не игнорировать кросс-валидацию — переобучение по-прежнему остается одной из главных проблем машинного обучения.